# EEGDash example for sex classification

The code below provides an example of using the *EEGDash* library in combination with PyTorch to develop a deep learning model for detecting sex in a collection of 136 subjects.

1. **Data Retrieval Using EEGDash**: An instance of *EEGDashDataset* is created to search and retrieve resting state data for 136 subjects (dataset ds005505). At this step, only the metadata is transferred.

2. **Data Preprocessing Using BrainDecode**: This process preprocesses EEG data using Braindecode by selecting specific channels, resampling, filtering, and extracting 2-second epochs. This takes about 2 minutes.

3. **Creating a train and testing sets**: The dataset is split into training (80%) and testing (20%) sets with balanced labels--making sure also that we have as many males as females--converted into PyTorch tensors, and wrapped in DataLoader objects for efficient mini-batch training.

4. **Model Definition**: The model is a custom convolutional neural network with 24 input channels (EEG channels), 2 output classes (male and female).

5. **Model Training and Evaluation Process**: This section trains the neural network, normalizes input data, computes cross-entropy loss, updates model parameters, and evaluates classification accuracy over six epochs. This takes less than 10 seconds to a couple of minutes, depending on the device you use.



## Data Retrieval Using EEGDash

First we find one resting state dataset for a collection of subject. The dataset ds005505 contains 136 subjects with both male and female participants.

In [1]:

# from eegdash import EEGDashDataset

# ds_sexdata = EEGDashDataset({'dataset': 'ds005505', 'task': 'RestingState'}, target_name='sex')

## Data Preprocessing Using Braindecode

[BrainDecode](https://braindecode.org/stable/install/install.html) is a specialized library for preprocessing EEG and MEG data. 

We apply three preprocessing steps in Braindecode:
1.	**Selection** of 24 specific EEG channels from the original 128.
2.	**Resampling** the EEG data to a frequency of 128 Hz.
3.	**Filtering** the EEG signals to retain frequencies between 1 Hz and 55 Hz.

When calling the **preprocess** function, the data is retrieved from the remote repository.

Finally, we use **create_windows_from_events** to extract 2-second epochs from the data. These epochs serve as the dataset samples.

In [2]:
# from braindecode.preprocessing import (preprocess, Preprocessor, create_fixed_length_windows)
# import os

# # Alternatively, if you want to include this as a preprocessing step in a Braindecode pipeline:
# preprocessors = [
#     Preprocessor('pick_channels', ch_names=['E22', 'E9', 'E33', 'E24', 'E11', 'E124', 'E122', 'E29', 'E6', 'E111', 'E45', 'E36', 'E104', 'E108', 'E42', 'E55', 'E93', 'E58', 'E52', 'E62', 'E92', 'E96', 'E70', 'Cz']),
#     Preprocessor("resample", sfreq=128),
#     Preprocessor("filter", l_freq=1, h_freq=55)
# ]
# preprocess(ds_sexdata, preprocessors, n_jobs=-1) #, save_dir='xxxx'' will save and set preload to false

# # extract windows and save to disk
# windows_ds = create_fixed_length_windows(ds_sexdata, start_offset_samples=0, stop_offset_samples=None,
#         window_size_samples=256, window_stride_samples=256, drop_last_window=True, preload=False) 
# os.makedirs('data/hbn_preprocessed_restingstate', exist_ok=True)
# windows_ds.save('data/hbn_preprocessed_restingstate', overwrite=True)

## Plotting a Single Channel for One Sample

It’s always a good practice to verify that the data has been properly loaded and processed. Here, we plot a single channel from one sample to ensure the signal is present and looks as expected.

In [3]:
# import matplotlib.pyplot as plt
# plt.figure()
# plt.plot(windows_ds[1000][0][0,:].transpose()) # first channel of first epoch
# plt.show()

## Load pre-saved data

If you have run the previous steps before, the data should be saved and may be reloaded here. If you are simply running this notebook for the first time, there is no need to reload the data, and this step may be skipped. However, it is quick, so you might as well execute the cell; it will have no consequences and will allow you to check that the data was saved properly.

In [4]:
from braindecode.datautil import load_concat_dataset

print("Loading data from disk")
windows_ds = load_concat_dataset(path='data/hbn_preprocessed_restingstate', preload=False)


Loading data from disk


## Feature Extraction

In [5]:
from eegdash.features import extract_features, FeatureExtractor
from eegdash.features.spectral import (
    SpectralFeatureExtractor,
    NormalizedSpectralFeatureExtractor,
    root_total_power,
    spectral_moment,
    spectral_entropy,
    spectral_edge,
    spectral_slope,
)
from functools import partial

features = FeatureExtractor(
    {
        "spec": SpectralFeatureExtractor(
            {
                "rtot_power": root_total_power,
                "slope": spectral_slope,
                "": NormalizedSpectralFeatureExtractor(
                    {
                        "moment": spectral_moment,
                        "entropy": spectral_entropy,
                        "edge": partial(spectral_edge, edge=0.8),
                    },
                    channel_names=windows_ds.datasets[0].raw.ch_names,
                )
            },
            channel_names=windows_ds.datasets[0].raw.ch_names,
            fs=windows_ds.datasets[0].raw.info['sfreq'],
        )
    },
)
features_ds = extract_features(windows_ds, features, n_jobs=-2)

In [6]:
import os

os.makedirs('data/hbn_features_restingstate', exist_ok=True)
features_ds.save('data/hbn_features_restingstate', overwrite=True)

In [7]:
from eegdash.features import load_features_concat_dataset

print("Loading features from disk")
features_ds = load_features_concat_dataset(path='data/hbn_features_restingstate', n_jobs=-2)

Loading features from disk


In [8]:
features_ds.to_dataframe()

,spec_rtot_power_E22,spec_rtot_power_E9,spec_rtot_power_E33,spec_rtot_power_E24,spec_rtot_power_E11,spec_rtot_power_E124,spec_rtot_power_E122,spec_rtot_power_E29,spec_rtot_power_E6,spec_rtot_power_E111,...,spec_edge_E55,spec_edge_E93,spec_edge_E58,spec_edge_E52,spec_edge_E62,spec_edge_E92,spec_edge_E96,spec_edge_E70,spec_edge_Cz,target
0,0.000115,0.000104,0.000097,0.000089,0.000085,0.000094,0.000096,0.000080,0.000030,0.000089,...,59.5,59.5,17.5,59.5,4.5,59.5,28.5,11.5,0.0,F
1,0.000052,0.000048,0.000044,0.000041,0.000040,0.000046,0.000045,0.000038,0.000015,0.000043,...,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,F
2,0.000048,0.000046,0.000042,0.000040,0.000038,0.000044,0.000044,0.000036,0.000014,0.000042,...,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,F
3,0.000047,0.000049,0.000044,0.000041,0.000039,0.000046,0.000046,0.000038,0.000015,0.000044,...,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,F
4,0.000050,0.000048,0.000044,0.000042,0.000040,0.000046,0.000046,0.000038,0.000015,0.000044,...,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28787,0.000096,0.000097,0.000097,0.000096,0.000096,0.000097,0.000097,0.000097,0.000097,0.000096,...,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,F
28788,0.000096,0.000097,0.000097,0.000097,0.000097,0.000097,0.000097,0.000097,0.000097,0.000096,...,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,F
28789,0.000096,0.000097,0.000097,0.000096,0.000096,0.000097,0.000098,0.000097,0.000097,0.000096,...,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,F
28790,0.000097,0.000097,0.000097,0.000097,0.000097,0.000097,0.000097,0.000097,0.000097,0.000096,...,60.0,60.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,F


## Creating a Training and Test Set

The code below creates a training and test set. We first split the data using the **train_test_split** function and then create a **TensorDataset** for both sets.

1. **Set Random Seed** – The random seed is fixed using `torch.manual_seed(random_state)` to ensure reproducibility in dataset splitting and model training.
2. **Get Balanced Indices for Male and Female Subjects** – We ensure a 50/50 split of male and female subjects in both the training and test sets. Additionally, we prevent subject leakage, meaning the same subjects do not appear in both sets. The dataset is split into training (90%) and testing (10%) subsets using `train_test_split()`, ensuring balanced stratification based on gender.
3. **Convert Data to PyTorch Tensors** – The selected training and testing samples are converted into `FloatTensor` for input features and `LongTensor` for labels, making them compatible with PyTorch models.
4. **Create DataLoaders** – The datasets are wrapped in PyTorch `DataLoader` objects with a batch size of 100, allowing efficient mini-batch training and shuffling. Although there are only 136 subjects, the dataset contains more than 10,000 2-second samples.


In [9]:
from eegdash.features import FeaturesConcatDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import numpy as np
import torch

# random seed for reproducibility
random_state = 0
np.random.seed(random_state)
torch.manual_seed(random_state)

# Get balanced indices for male and female subjects and create a balanced dataset
male_subjects   = features_ds.description['subject'][features_ds.description['sex'] == 'M']
female_subjects = features_ds.description['subject'][features_ds.description['sex'] == 'F']
n_samples = min(len(male_subjects), len(female_subjects))
balanced_subjects = np.concatenate([male_subjects[:n_samples], female_subjects[:n_samples]])
balanced_gender = ['M'] * n_samples + ['F'] * n_samples
train_subj, val_subj, train_gender, val_gender = train_test_split(balanced_subjects, balanced_gender, train_size=0.9, stratify=balanced_gender, random_state=random_state)

# Create datasets
train_ds = FeaturesConcatDataset([ds for ds in features_ds.datasets if ds.description.subject in train_subj])
val_ds = FeaturesConcatDataset([ds for ds in features_ds.datasets if ds.description.subject in val_subj])

# Create dataloaders
train_loader = DataLoader(train_ds, batch_size=100, shuffle=True)
val_loader =  DataLoader(val_ds, batch_size=100, shuffle=True)
# Check the balance of the dataset
assert len(balanced_subjects) == len(balanced_gender)
print(f"Number of subjects in balanced dataset: {len(balanced_subjects)}")
print(f"Gender distribution in balanced dataset: {np.unique(balanced_gender, return_counts=True)}")

Number of subjects in balanced dataset: 92
Gender distribution in balanced dataset: (array(['F', 'M'], dtype='<U1'), array([46, 46]))


# Check labels

It is good practice to verify the labels and ensure the random seed is functioning correctly. If all labels are 'M' (male) or 'F' (female), it could indicate an issue with data loading or stratification, requiring further investigation.

In [10]:
# get the first batch to check the labels
dataiter = iter(train_loader)
first_item, label = dataiter.__next__()
np.array(label).T

array(['F', 'F', 'M', 'F', 'F', 'M', 'M', 'M', 'M', 'F', 'M', 'M', 'F',
       'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'F', 'F', 'M',
       'M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'F', 'M', 'F',
       'F', 'M', 'M', 'F', 'F', 'M', 'M', 'F', 'M', 'M', 'F', 'M', 'F',
       'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'M',
       'F', 'F', 'F', 'M', 'M', 'M', 'F', 'F', 'F', 'M', 'F', 'M', 'M',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'M', 'M', 'M', 'F', 'M', 'F',
       'F', 'M', 'F', 'F', 'F', 'F', 'F', 'F', 'M'], dtype='<U1')

# Create model

The model is a custom convolutional neural network with 24 input channels (EEG channels), 2 output classes (male vs. female), and an input window size of 256 samples (2 seconds of EEG data). See the reference below for more information.

[1] Truong, D., Milham, M., Makeig, S., & Delorme, A. (2021). Deep Convolutional Neural Network Applied to Electroencephalography: Raw Data vs Spectral Features. IEEE Engineering in Medicine and Biology Society. Annual International Conference, 2021, 1039–1042. https://doi.org/10.1109/EMBC46164.2021.9630708



In [11]:
# create model
from torchinfo import summary
from torch import nn

# MLP
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(features_ds.datasets[0].n_features, 100),
    nn.Linear(100, 100),
    nn.Linear(100, 2),
)

# print(summary(model, input_size=(1, 1, 24, 256)))
print(summary(model, input_size=first_item.shape))


Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [100, 2]                  --
├─Flatten: 1-1                           [100, 144]                --
├─Linear: 1-2                            [100, 100]                14,500
├─Linear: 1-3                            [100, 100]                10,100
├─Linear: 1-4                            [100, 2]                  202
Total params: 24,802
Trainable params: 24,802
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.48
Input size (MB): 0.06
Forward/backward pass size (MB): 0.16
Params size (MB): 0.10
Estimated Total Size (MB): 0.32


# Model Training and Evaluation Process

This section trains the neural network using the Adamax optimizer, normalizes input data, computes cross-entropy loss, updates model parameters, and tracks accuracy across six epochs.

1. **Set Up Optimizer and Learning Rate Scheduler** – The `Adamax` optimizer initializes with a learning rate of 0.002 and weight decay of 0.001 for regularization.  

2. **Allocate Model to Device** – The model moves to the specified device (CPU, GPU, or MPS for Mac silicon) to optimize computation efficiency.  

3. **Normalize Input Data** – The `normalize_data` function standardizes input data by subtracting the mean and dividing by the standard deviation along the time dimension before transferring it to the appropriate device.  

4. **Train the Model for Two Epochs** – The training loop iterates through data batches with the model in training mode. It normalizes inputs, computes predictions, calculates cross-entropy loss, performs backpropagation, updates model parameters, and steps the learning rate scheduler. It tracks correct predictions to compute accuracy.  

5. **Evaluate on Test Data** – After each epoch, the model runs in evaluation mode on the test set. It computes predictions on normalized data and calculates test accuracy by comparing outputs with actual labels.  


In [12]:

from torch.nn import functional as F

optimizer = torch.optim.Adamax(model.parameters(), lr=0.002, weight_decay=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
model.to(device=device)

def normalize_data(x):
    x = x.reshape(x.shape[0], -1)
    mean = x.mean(dim=1, keepdim=True)
    std = x.std(dim=1, keepdim=True) + 1e-7  # add small epsilon for numerical stability
    x = (x - mean) / std
    x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
    return x

# dictionary of genders for converting sample labels to numerical values
gender_dict = {'M': 0, 'F': 1}

epochs = 2
for e in range(epochs):
    
    # training
    correct_train = 0
    for t, (x, y) in enumerate(train_loader):
        model.train()  # put model to training mode
        # scores = model(normalize_data(x))
        scores = model(x)
        _, preds = scores.max(1)
        y = torch.tensor([gender_dict[gender] for gender in y], device=device, dtype=torch.long)
        correct_train += (preds == y).sum()/len(train_ds)

        # Calculates the cross-entropy loss and performs backpropagation
        loss = F.cross_entropy(scores, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if t % 50 == 0:
            print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
    
    # validation
    correct_test = 0
    for t, (x, y) in enumerate(val_loader):
        model.eval() # put model to testing mode
        scores = model(normalize_data(x))
        _, preds = scores.max(1)
        y = torch.tensor([gender_dict[gender] for gender in y], device=device, dtype=torch.long)
        correct_test += (preds == y).sum()/len(val_ds)

    print(f'Epoch {e}, Train accuracy: {correct_train:.2f}, Test accuracy: {correct_test:.2f}\n')


Epoch 0, Iteration 0, loss = nan
Epoch 0, Iteration 50, loss = nan
Epoch 0, Iteration 100, loss = nan
Epoch 0, Iteration 150, loss = nan
Epoch 0, Train accuracy: 0.51, Test accuracy: 0.57

Epoch 1, Iteration 0, loss = nan
Epoch 1, Iteration 50, loss = nan
Epoch 1, Iteration 100, loss = nan
Epoch 1, Iteration 150, loss = nan
Epoch 1, Train accuracy: 0.51, Test accuracy: 0.57

